# Exact cover

Exact cover algorithm implemented by QUBO in quantum annealer

In [1]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector

## Define sets

In [36]:
U = [0,1,2,3,4,5,6,7,8,9]
V = [[0,1,2], [2,3], [6,7,8,9], [7,9], [4,5,6], [4,5], [9]]

def result_info(sampleset):
    r = sampleset.first.sample
    r = [V[k] for k, v in r.items() if v==1]
    print('Edges: ' + str(r))

## Create qubo

Constraints
- total number of elements in subset should be (len(U))
- each element only in one subset

In [28]:
Q = np.zeros((len(V),len(V)))

# Total elements constraint

max_count = 0

for i in range(len(V)):
    Q[i][i] =- len(V[i])
    max_count += len(V[i])

# each element only in one subset

for a in U:
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            if a in V[i] and a in V[j]:
                Q[i][j] = max_count
print(Q)

[[-3. 17.  0.  0.  0.  0.  0.]
 [ 0. -2.  0.  0.  0.  0.  0.]
 [ 0.  0. -4. 17. 17.  0. 17.]
 [ 0.  0.  0. -2.  0.  0. 17.]
 [ 0.  0.  0.  0. -3. 17.  0.]
 [ 0.  0.  0.  0.  0. -2.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.]]


## Creat BQM from QUBO

In [29]:
bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
#bqm = bqm.relabel_variables(edge_names, inplace=False)

# Local deterministic solver

In [30]:
sampleset = dimod.ExactSolver().sample(bqm)
print(sampleset.truncate(10))

   0  1  2  3  4  5  6 energy num_oc.
0  1  0  1  0  0  1  0   -9.0       1
1  0  1  1  0  0  1  0   -8.0       1
2  1  0  0  1  1  0  0   -8.0       1
3  1  0  0  1  0  1  0   -7.0       1
4  1  0  0  0  1  0  1   -7.0       1
5  1  0  1  0  0  0  0   -7.0       1
6  0  1  0  1  1  0  0   -7.0       1
7  0  1  1  0  0  0  0   -6.0       1
8  0  0  1  0  0  1  0   -6.0       1
9  1  0  0  0  0  1  1   -6.0       1
['BINARY', 10 rows, 10 samples, 7 variables]


In [37]:
result_info(sampleset)

Edges: [[0, 1, 2], [6, 7, 8, 9], [4, 5]]


## Local heuristic classical solver

In [40]:
sampleset2 = SimulatedAnnealingSampler().sample(bqm, num_reads=500)
print(sampleset2.aggregate().truncate(10))

   0  1  2  3  4  5  6 energy num_oc.
0  1  0  1  0  0  1  0   -9.0     164
1  0  1  1  0  0  1  0   -8.0      63
2  1  0  0  1  1  0  0   -8.0      72
3  1  0  0  0  1  0  1   -7.0      27
4  1  0  0  1  0  1  0   -7.0      27
5  1  0  1  0  0  0  0   -7.0      23
6  0  1  0  1  1  0  0   -7.0      25
7  1  0  0  0  0  1  1   -6.0      10
8  0  1  1  0  0  0  0   -6.0      10
9  0  1  0  1  0  1  0   -6.0      12
['BINARY', 10 rows, 433 samples, 7 variables]


In [39]:
result_info(sampleset)

Edges: [[0, 1, 2], [6, 7, 8, 9], [4, 5]]
